In [1]:
import numpy as np
from numpy.typing import NDArray
from ska_sdp_datamodels.visibility import Visibility as RASCILVisibility
from typing_extensions import assert_never

from karabo.imaging.imager_base import DirtyImagerConfig
from karabo.imaging.imager_oskar import OskarDirtyImager
from karabo.imaging.imager_rascil import RascilDirtyImager,RascilDirtyImagerConfig
from karabo.imaging.util import (
    auto_choose_dirty_imager_from_sim,
    auto_choose_dirty_imager_from_vis,
)
from karabo.imaging.imager_wsclean import ( # type: ignore
    WscleanDirtyImager,
    WscleanImageCleaner,
    WscleanImageCleanerConfig,
    create_image_custom_command,
)
from karabo.imaging import imager_wsclean
from karabo.simulation.interferometer import InterferometerSimulation
from karabo.simulation.observation import Observation
from karabo.simulation.sky_model import SkyModel
from karabo.simulation.telescope import Telescope
from karabo.simulation.visibility import Visibility
from karabo.imaging.imager_oskar import OskarDirtyImager, OskarDirtyImagerConfig
from karabo.simulator_backend import SimulatorBackend
from astropy.io import fits
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import matplotlib as mpl
from karabo.imaging.image import ImageMosaicker
mpl.use('Tkagg')

from mpl_toolkits.axes_grid1 import make_axes_locatable
from astropy.io import fits
from astropy.wcs import WCS
from astropy import units as u

The RASCIL data directory is not available - continuing but any simulations will fail


In [3]:
############# Code Cell for SKA-mid and Meerkat Configuration ####################
telescope_path = '/media/rohit/sdata/ska-solar-files/meerkat.tm'
#telescope_path = '/media/rohit/sdata/ska-solar-files/ska1mid.tm'
file_name='mightee_meerkat_snapshot0'
noise_enable_ = False
enable_array_beam=False
#path = '/media/rohit/sdata/karabo-paper-files/sim/5h_meerkat'
#path = '/media/rohit/sdata/karabo-paper-files/sim/5h_skamid'
path = '/media/rohit/sdata/karabo-paper-files/sim/snapshot_meerkat'
#path = '/media/rohit/sdata/karabo-paper-files/sim/snapshot_skamid'
print('#### TELESCOPE PATH: ',telescope_path)
print('#### File Name: ',file_name)
sky = SkyModel()
mightee1 = SkyModel.get_MIGHTEE_Sky()
mightee_continuum = mightee1.to_np_array()
sky_data = np.zeros((len(mightee_continuum), 12))
sky_data[:, 0] = mightee_continuum[:, 0]
sky_data[:, 1] = mightee_continuum[:, 1]
sky_data[:, 2] = mightee_continuum[:, 2]
sky_data[:, 6] = mightee_continuum[:, 6]
sky_data[:, 9] = mightee_continuum[:, 9]
sky_data[:, 10] = mightee_continuum[:, 10]
sky_data[:, 11] = mightee_continuum[:, 11]
sky.add_point_sources(sky_data)
ra_list = [150.0];dec_list = [2.0];f_obs = 8.56e8;chan = 0.2e6 # at SKA-mid frequencies
sec_ = 8;t_int = 8; nchan =4096 # for SKA mid freq
hours_= 0
min_= 0 # 1 hour pointing
t_nchan = int((hours_*3600.0+min_*60.+sec_)/t_int)

print(' Number of sky sources:',sky_data.shape[0],' Number of time channels:',t_nchan,' Number of freqeuncy channels:',nchan)
#---------------------------------
layout=np.loadtxt(telescope_path+'/layout.txt')
nant=len(layout)
nb=int(nant*(nant-1)*0.5)
print('Number of Baselines:',nb)
base_length=[0]*nb
k=0
for i in range(nant):
    for j in range(i,nant):
        if(i!=j):
            base_length[k] = np.sqrt((layout[i][0]-layout[j][0])**2 + (layout[i][1]-layout[j][1])**2)
            k=k+1
base_length=np.array(base_length)
beam_size_arcsec = 3.e8/f_obs/base_length.max()*180/np.pi*3600
print('Maximum Baseline (m)):',base_length.max(),' Beam Size (arcsec):',beam_size_arcsec)
img_nsize = 4096
cellsize_arcsec=3 #beam_size_arcsec/3 # in arcsec
cellsize_rad = cellsize_arcsec/3600.*np.pi/180.
print('BEAM SIZE: ',beam_size_arcsec,' arcsec',' | IMAGE CELL SIZE: ',cellsize_arcsec,' arcsec','| IMG SIZE:',img_nsize,' pixels','IMG FOV',img_nsize*cellsize_arcsec/3600.,' deg')
print('##---------- RA-DEC Iteration begins -----------##')
k=0
for phase_ra in ra_list:
    for phase_dec in dec_list:
        print('Iteration:',k,'| R.A.:',phase_ra,'deg | DEC:',phase_dec,'deg')
        sky_filter = sky.filter_by_radius(
            ra0_deg=phase_ra,
            dec0_deg=phase_dec,
            inner_radius_deg=0,
            outer_radius_deg=100.0,
        )
        # Skip analysis if there are no sources in this iteration
        if sky_filter.sources is None:
            continue
        
        telescope=Telescope.read_OSKAR_tm_file(telescope_path)
        telescope.read_OSKAR_tm_file(telescope_path)
        simulation = InterferometerSimulation(
            channel_bandwidth_hz=chan,
            time_average_sec=1,
            noise_enable=False,
            noise_seed="time",
            noise_freq="Range",
            noise_rms="Range",
            noise_start_freq=f_obs,
            noise_inc_freq=chan,
            noise_number_freq=1,
            noise_rms_start=5,
            noise_rms_end=10,
            ms_file_path=path+'/'+file_name+str(k)+'.ms',
            vis_path=path+'/'+file_name+str(k)+'.vis'
        )
        observation = Observation(
            phase_centre_ra_deg=phase_ra,
            start_date_and_time=datetime(2022, 9, 1, 9, 00, 00, 521489),
            length=timedelta(hours=hours_, minutes=min_, seconds=sec_, milliseconds=0),
            phase_centre_dec_deg=phase_dec,
            number_of_time_steps=t_nchan,
            start_frequency_hz=f_obs,
            frequency_increment_hz=chan,
            number_of_channels=nchan,
        )
        print('--- Simulation Run Begins....')
        visibility = simulation.run_simulation(
            telescope, sky, observation
        )
        print('--- Simulation Run Ends....')
        k=k+1
        
"""         dirty_imager = RascilDirtyImager(
            RascilDirtyImagerConfig(
                imaging_npixel=img_nsize,
                imaging_cellsize=cellsize_rad,
                combine_across_frequencies=False,
            )
        )
        
        dirty = dirty_imager.create_dirty_image(visibility)

        dirty.write_to_file(
            path+'mightee_'+file_name+str(k)+'.fits',
            overwrite=True,
        ) """


#### TELESCOPE PATH:  /media/rohit/sdata/ska-solar-files/meerkat.tm
#### File Name:  mightee_meerkat_snapshot0
 Number of sky sources: 9896  Number of time channels: 1  Number of freqeuncy channels: 4096
Number of Baselines: 2016
Maximum Baseline (m)): 7697.525274944539  Beam Size (arcsec): 9.391208866213761
BEAM SIZE:  9.391208866213761  arcsec  | IMAGE CELL SIZE:  3  arcsec | IMG SIZE: 4096  pixels IMG FOV 3.4133333333333336  deg
##---------- RA-DEC Iteration begins -----------##
Iteration: 0 | R.A.: 150.0 deg | DEC: 2.0 deg
Parameter 'use_gpus' is None! Using function 'karabo.util.gpu_util.is_cuda_available()'. To overwrite, set 'use_gpus' True or False.
--- Simulation Run Begins....


KeyboardInterrupt: 

In [3]:
############# Code Cell for SKA-low and MWA Configuration ####################
telescope_path = '/media/rohit/sdata/ska-solar-files/mwa.phase1.tm'
#telescope_path = '/media/rohit/sdata/ska-solar-files/ska1low.tm'
file_name='gleam_mwa_long0'
noise_enable_ = False
enable_array_beam=False
#path = '/media/rohit/sdata/karabo-paper-files/sim/5h_mwa'
#path = '/media/rohit/data/karabo-paper-files/sim/5h_skalow'
path = '/media/rohit/sdata/karabo-paper-files/sim/snapshot_mwa'
#path = '/media/rohit/sdata/karabo-paper-files/sim/snapshot_skalow'
print('#### TELESCOPE PATH: ',telescope_path)
print('#### File Name: ',file_name)


#### TELESCOPE PATH:  /media/rohit/sdata/ska-solar-files/mwa.phase1.tm
#### File Name:  gleam_mwa_long0


In [14]:

sky = SkyModel()
gleam1 = SkyModel.get_GLEAM_Sky()
gleam_continuum = gleam1.to_np_array()
sky_data = np.zeros((len(gleam_continuum), 12))
sky_data[:, 0] = gleam_continuum[:, 0]
sky_data[:, 1] = gleam_continuum[:, 1]
sky_data[:, 2] = gleam_continuum[:, 2]
sky_data[:, 6] = gleam_continuum[:, 6]
sky_data[:, 9] = gleam_continuum[:, 9]
sky_data[:, 10] = gleam_continuum[:, 10]
sky_data[:, 11] = gleam_continuum[:, 11]
sky.add_point_sources(sky_data)
ra_list = [292.5];dec_list = [-55.0];f_obs = 72.0e6;chan = 40.0e3*256
phase_ra = ra_list[0];phase_dec = dec_list[0]
sky_filter = sky.filter_by_radius(
    ra0_deg=phase_ra,
    dec0_deg=phase_dec,
    inner_radius_deg=0,
    outer_radius_deg=6.0,
)
sky_filter_data = sky_filter.to_np_array()
#sec_ = 8;t_int = 8; nchan =4096 # for SKA mid freq
sec_ = 0.5;t_int = 10; nchan=int(1536/256.)# for SKA low freq
hours_= 0
min_= 1 # 1 hour pointing
t_nchan = int((hours_*3600.0+min_*60.+sec_)/t_int)
print(' Number of sky sources:',sky_filter_data.shape[0],' Number of time channels:',t_nchan,' Number of freqeuncy channels:',nchan)
#---------------------------------
layout=np.loadtxt(telescope_path+'/layout.txt')
nant=len(layout)
nb=int(nant*(nant-1)*0.5)
print('Number of Baselines:',nb)
base_length=[0]*nb
k=0
for i in range(nant):
    for j in range(i,nant):
        if(i!=j):
            base_length[k] = np.sqrt((layout[i][0]-layout[j][0])**2 + (layout[i][1]-layout[j][1])**2)
            k=k+1
base_length=np.array(base_length)
beam_size_arcsec = 3.e8/f_obs/base_length.max()*180/np.pi*3600
print('Maximum Baseline (m)):',base_length.max(),' Beam Size (arcsec):',beam_size_arcsec)
img_nsize = 4096
cellsize_arcsec=30  #beam_size_arcsec/3 # in arcsec
cellsize_rad = cellsize_arcsec/3600.*np.pi/180.
print('BEAM SIZE: ',beam_size_arcsec,' arcsec',' | IMAGE CELL SIZE: ',cellsize_arcsec,' arcsec','| IMG SIZE:',img_nsize,' pixels','IMG FOV',img_nsize*cellsize_arcsec/3600.,' deg')


 Number of sky sources: 23600  Number of time channels: 6  Number of freqeuncy channels: 6
Number of Baselines: 8128
Maximum Baseline (m)): 2873.5020030311443  Beam Size (arcsec): 299.0903405634132
BEAM SIZE:  299.0903405634132  arcsec  | IMAGE CELL SIZE:  30  arcsec | IMG SIZE: 4096  pixels IMG FOV 34.13333333333333  deg


In [15]:

print('##---------- RA-DEC Iteration begins -----------##')
k=0
for phase_ra in ra_list:
    for phase_dec in dec_list:
        print('Iteration:',k,'| R.A.:',phase_ra,'deg | DEC:',phase_dec,'deg')
        # Skip analysis if there are no sources in this iteration
        if sky_filter.sources is None:
            continue
        telescope=Telescope.read_OSKAR_tm_file(telescope_path)
        telescope.read_OSKAR_tm_file(telescope_path)
        simulation = InterferometerSimulation(
            channel_bandwidth_hz=chan,
            time_average_sec=1,
            noise_enable=False,
            noise_seed="time",
            noise_freq="Range",
            noise_rms="Range",
            noise_start_freq=f_obs,
            noise_inc_freq=chan,
            noise_number_freq=1,
            noise_rms_start=5,
            noise_rms_end=10,
            ms_file_path=path+'/'+file_name+str(k)+'.ms',
            #vis_path=path+'/'+file_name+str(k)+'.vis'
        )
        observation = Observation(
            phase_centre_ra_deg=phase_ra,
            start_date_and_time=datetime(2022, 9, 1, 9, 00, 00, 521489),
            length=timedelta(hours=hours_, minutes=min_, seconds=sec_, milliseconds=0),
            phase_centre_dec_deg=phase_dec,
            number_of_time_steps=t_nchan,
            start_frequency_hz=f_obs,
            frequency_increment_hz=chan,
            number_of_channels=nchan,
        )
        print('--- Simulation Run Begins....')
        visibility = simulation.run_simulation(
            telescope, sky_filter, observation
        )
        print('--- Simulation Run Ends....')
        k=k+1
        
"""         dirty_imager = RascilDirtyImager(
            RascilDirtyImagerConfig(
                imaging_npixel=img_nsize,
                imaging_cellsize=cellsize_rad,
                combine_across_frequencies=False,
            )
        )
        
        dirty = dirty_imager.create_dirty_image(visibility)

        dirty.write_to_file(
            path+'mightee_'+file_name+str(k)+'.fits',
            overwrite=True,
        ) """


##---------- RA-DEC Iteration begins -----------##
Iteration: 0 | R.A.: 292.5 deg | DEC: -55.0 deg
Parameter 'use_gpus' is None! Using function 'karabo.util.gpu_util.is_cuda_available()'. To overwrite, set 'use_gpus' True or False.
--- Simulation Run Begins....
Creating /tmp/karabo-STM-rohit-TRnaSBphUX/interferometer-61ZFcxcuCg for interferometer disk-cache.
Saved visibility to /tmp/karabo-STM-rohit-TRnaSBphUX/interferometer-61ZFcxcuCg/visibility.vis
--- Simulation Run Ends....


"         dirty_imager = RascilDirtyImager(\n            RascilDirtyImagerConfig(\n                imaging_npixel=img_nsize,\n                imaging_cellsize=cellsize_rad,\n                combine_across_frequencies=False,\n            )\n        )\n        \n        dirty = dirty_imager.create_dirty_image(visibility)\n\n        dirty.write_to_file(\n            path+'mightee_'+file_name+str(k)+'.fits',\n            overwrite=True,\n        ) "

In [12]:
sky_filter.to_np_array().shape

(67180, 14)

In [17]:
nchan_img = nchan
k=0
print('----OUTPUT MS PATH:',visibility.ms_file_path,'-----------')
output_img = path+'/img/'+file_name+str(k)+'.fits'
print('----OUTPUT IMG PATH:',output_img,'-----------')
print('##### IMAGING Begins.....')
img_cmd = 'wsclean \
        -size '+str(img_nsize)+' '+str(img_nsize)+' \
        -name '+output_img+' \
        -scale '+str(cellsize_rad)+'rad -niter 5000 -mgain 0.8 \
        -channels-out '+str(nchan_img)+' '+visibility.ms_file_path
print(img_cmd)
restored = imager_wsclean.create_image_custom_command(command=img_cmd)

----OUTPUT MS PATH: /media/rohit/sdata/karabo-paper-files/sim/snapshot_mwa/gleam_mwa_long00.ms -----------
----OUTPUT IMG PATH: /media/rohit/sdata/karabo-paper-files/sim/snapshot_mwa/img/gleam_mwa_long00.fits -----------
##### IMAGING Begins.....
wsclean         -size 4096 4096         -name /media/rohit/sdata/karabo-paper-files/sim/snapshot_mwa/img/gleam_mwa_long00.fits         -scale 0.0001454441043328608rad -niter 5000 -mgain 0.8         -channels-out 6 /media/rohit/sdata/karabo-paper-files/sim/snapshot_mwa/gleam_mwa_long00.ms
Creating /tmp/karabo-STM-rohit-TRnaSBphUX/WSClean-custom-9oWGeITvDX for Disk cache for WSClean custom command images
WSClean command: [cd /tmp/karabo-STM-rohit-TRnaSBphUX/WSClean-custom-9oWGeITvDX && OPENBLAS_NUM_THREADS=1 wsclean         -size 4096 4096         -name /media/rohit/sdata/karabo-paper-files/sim/snapshot_mwa/img/gleam_mwa_long00.fits         -scale 0.0001454441043328608rad -niter 5000 -mgain 0.8         -channels-out 6 /media/rohit/sdata/karabo-p

FileNotFoundError: [Errno 2] No such file or directory: '/tmp/karabo-STM-rohit-TRnaSBphUX/WSClean-custom-9oWGeITvDX/wsclean-image.fits'

In [ ]:



path = '/media/rohit/sdata/karabo-paper-files/sim/snapshot_meerkat'
file_name = 'meerkat'
d,h=fits.getdata(path+'/img/mightee_'+file_name+'0.fits',header=True)
#d,h=fits.getdata(path+'wsclean_img.fits',header=True)
d=d[0,0]
nsize = d.shape[0]
delt=h['CDELT1']
l1 = h['CRVAL1'] + h['CDELT1']*nsize
l2 = h['CRVAL1'] - h['CDELT1']*nsize
r1 = h['CRVAL2'] + h['CDELT2']*nsize
r2 = h['CRVAL2'] - h['CDELT2']*nsize
npix_per_beam = beam_size_arcsec/cellsize_arcsec
wcs = WCS(h)
img_rms = d[0:300,0:300].std()
img_max = d.max()
img_dr = img_max/img_rms
print(img_rms, img_max, img_dr)

In [ ]:
mpl.use('Tkagg')
d1,h=fits.getdata('/media/rohit/sdata/karabo-paper-files/sim/5h_meerkat/img/mightee_meerkat_long01.fits-image.fits',header=True)
d=d1[0,0]
nsize = d[0].shape[0]
delt=h['CDELT1']
wcs = WCS(h)
print(nsize,delt, wcs.pixel_n_dim)
ra_grid,dec_grid=np.meshgrid((np.arange(nsize)-int(nsize/2))*delt,(np.arange(nsize)-int(nsize/2))*delt)
ra_grid = ra_grid + h['CRVAL1']
dec_grid = dec_grid + h['CRVAL2']

f,ax0=plt.subplots(1,1,subplot_kw={'projection': wcs,'slices':('x','y',0,0)})
im0=ax0.imshow(d,origin='lower',
               cmap='YlGnBu',norm=mpl.colors.LogNorm(vmin=0.00005))
divider = make_axes_locatable(ax0)
cax = divider.append_axes('right', size='5%', pad=0.0)
caxx=cax.coords[0]
caxy=cax.coords[1]
caxx.grid(False)
caxy.grid(False)
cax.tick_params(direction='out')
caxx.set_ticks(alpha=0, color='w', size=0, values=[0]*u.dimensionless_unscaled)
caxy.set_ticklabel_position('r')
caxx.set_auto_axislabel(False)
caxy.set_auto_axislabel(False)
cax.set_title('Flux Density (Jy)')
f.colorbar(im0, cax=cax, orientation='vertical')
ax0.set_xlabel('R.A. (Deg)');ax0.set_ylabel('DEC (deg)')
plt.show()


In [ ]:

mpl.use('Tkagg')
f,ax1=plt.subplots(1,1,subplot_kw={'projection': wcs,'slices':('x','y',0,0)})
cm = plt.cm.get_cmap('Blues')
im1=ax1.scatter(sky_data[:, 0],sky_data[:, 1],c=sky_data[:,2], s=sky_data[:,2]*10000, vmax=0.01, cmap=cm,alpha=0.6)
divider = make_axes_locatable(ax1)
cax = divider.append_axes('right', size='10%', pad=0.2)
cax.coords[0].grid(False)
cax.coords[1].grid(False)
cax.tick_params(direction='in')
cax.coords[0].set_ticks(alpha=0, color='w', size=0, values=[]*u.dimensionless_unscaled)
cax.coords[1].set_ticklabel_position('r')
cax.coords[1].set_axislabel_position('r')
f.colorbar(im1, cax=cax, orientation='vertical',label = 'Jy')
ax1.set_xlabel('R.A. (Deg)');ax1.set_ylabel('DEC (deg)')
ax1.invert_xaxis()
plt.show()

GLEAM Simulation Ahead 

In [19]:
sky = SkyModel()
gleam1 = SkyModel.get_GLEAM_Sky()
gleam_continuum = gleam1.to_np_array()
sky_data = np.zeros((len(gleam_continuum), 12))
sky_data[:, 0] = gleam_continuum[:, 0]
sky_data[:, 1] = gleam_continuum[:, 1]
sky_data[:, 2] = gleam_continuum[:, 2]
sky_data[:, 6] = gleam_continuum[:, 6]
sky_data[:, 9] = gleam_continuum[:, 9]
sky_data[:, 10] = gleam_continuum[:, 10]
sky_data[:, 11] = gleam_continuum[:, 11]
sky.add_point_sources(sky_data)
ra_list = [292.5];dec_list = [-55.0];f_obs = 72.0e6;chan = 40.0e3*256
phase_ra = ra_list[0];phase_dec = dec_list[0]
sky_filter = sky.filter_by_radius(
    ra0_deg=phase_ra,
    dec0_deg=phase_dec,
    inner_radius_deg=0,
    outer_radius_deg=6.0,
)
data_gleam = sky_filter.to_np_array()
mpl.use('Tkagg')


In [22]:
mpl.use('Tkagg')
d1,h=fits.getdata('/media/rohit/sdata/karabo-paper-files/sim/snapshot_mwa/img/gleam_mwa_long00.fits-0000-image.fits',header=True)
d=d1[0,0]
nsize = d[0].shape[0]
delt=h['CDELT1']
wcs = WCS(h)
print(nsize,delt, wcs.pixel_n_dim)
ra_grid,dec_grid=np.meshgrid((np.arange(nsize)-int(nsize/2))*delt,(np.arange(nsize)-int(nsize/2))*delt)
ra_grid = ra_grid + h['CRVAL1']
dec_grid = dec_grid + h['CRVAL2']



f,ax1=plt.subplots(1,1,subplot_kw={'projection': wcs,'slices':('x','y',0,0)})
cm = plt.cm.get_cmap('Blues')
im1=ax1.scatter(data_gleam[:, 0],data_gleam[:, 1],c=data_gleam[:,2], s=data_gleam[:,2]*10, vmax=0.01, cmap=cm,alpha=0.6)
divider = make_axes_locatable(ax1)
cax = divider.append_axes('right', size='5%', pad=0.0)
caxx=cax.coords[0]
caxy=cax.coords[1]
caxx.grid(False)
caxy.grid(False)
cax.tick_params(direction='out')
caxx.set_ticks(alpha=0, color='w', size=0, values=[0]*u.dimensionless_unscaled)
caxy.set_ticklabel_position('r')
caxx.set_auto_axislabel(False)
caxy.set_auto_axislabel(False)
cax.set_title('Flux Density (Jy)')
f.colorbar(im1, cax=cax, orientation='vertical',label = 'Jy')
ax1.set_xlabel('R.A. (Deg)');ax1.set_ylabel('DEC (deg)')
ax1.invert_xaxis()
plt.show()

4096 -0.00833333333333333 4


/tmp/ipykernel_971250/292406991.py:15: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cm = plt.cm.get_cmap('Blues')
/home/rohit/miniconda3/envs/karabo-dev-env/lib/python3.9/site-packages/matplotlib/collections.py:996: RuntimeWarning: invalid value encountered in sqrt
  scale = np.sqrt(self._sizes) * dpi / 72.0 * self._factor
/home/rohit/miniconda3/envs/karabo-dev-env/lib/python3.9/site-packages/matplotlib/collections.py:996: RuntimeWarning: invalid value encountered in sqrt
  scale = np.sqrt(self._sizes) * dpi / 72.0 * self._factor
/home/rohit/miniconda3/envs/karabo-dev-env/lib/python3.9/site-packages/matplotlib/collections.py:996: RuntimeWarning: invalid value encountered in sqrt
  scale = np.sqrt(self._sizes) * dpi / 72.0 * self._factor
/home/rohit/miniconda3/envs/karabo-dev-env/lib/python3.9/site-packages/mat

In [26]:


f,ax0=plt.subplots(1,1,subplot_kw={'projection': wcs,'slices':('x','y',0,0)})
im0=ax0.imshow(d,origin='lower',
               cmap='YlGnBu',norm=mpl.colors.LogNorm(vmin=0.05))
divider = make_axes_locatable(ax0)
cax = divider.append_axes('right', size='5%', pad=0.0)
caxx=cax.coords[0]
caxy=cax.coords[1]
caxx.grid(False)
caxy.grid(False)
cax.tick_params(direction='out')
caxx.set_ticks(alpha=0, color='w', size=0, values=[0]*u.dimensionless_unscaled)
caxy.set_ticklabel_position('r')
caxx.set_auto_axislabel(False)
caxy.set_auto_axislabel(False)
cax.set_title('Flux Density (Jy)')
f.colorbar(im0, cax=cax, orientation='vertical')
ax0.set_xlabel('R.A. (Deg)');ax0.set_ylabel('DEC (deg)')
plt.show()


In [ ]:
#telescope.plot_telescope()
#plt.show()

In [15]:
sources_array = np.array([517220,261180,67180,16440,6460])
sources_array_run = np.array([833,433,109,29,12])

fchannel_array = np.array([1536,3072,6144,9216,12288])
fchannel_array_run = np.array([12,22,45,71,89])

tchannel_array = np.array([1200,3600,7200, 14400, 36000])
tchannel_array_run = np.array([1,1.5,2.1,2.8,4.9])


f,ax=plt.subplots(1,1)
ax.plot(sources_array_run,sources_array,'o-',color='r',label = 'Source Scaling')
ax.set_ylabel('Number of Sources / Frequency Channels / Time Channels')
ax.set_xlabel('Computation Time (min)')
#ax1 = ax.twinx()
ax.plot(fchannel_array_run,fchannel_array,'o-',color='g',label = 'Frequency Channel Scaling')
#ax2 = ax.twinx()
ax.plot(tchannel_array_run,tchannel_array,'o-',color='b',label = 'Time Channel Scaling')
ax.legend()
#ax1.legend()
plt.show()

In [ ]:
s = SkyModel.get_GLEAM_Sky()
#SkyModel.get_GLEAM_Sky?
kwargs={'xticks':'2,3,4,5','yticks':'45,50,55,60'}
s.explore_sky([250,-80],s=0.1,xlabel='RA (deg)',ylabel='DEC (deg)',
              cmap='plasma',cbar_label='log$_{10}$(S(Jy))')
              #xlim = (-1.5, 1.5),ylim = (-1.5, 1.5),
plt.xticks([2,3,4,5],[2,3,4,50])
plt.yticks([45,50,55,60])            
plt.show()